# The K-profile parameterization in `OceanTurb.jl`

Here we demonstrate how to set up a `Model` in `OceanTurb.jl`, set initial conditions and boundary conditions, and iterate the model forward.

In [ ]:
using Pkg; Pkg.activate(".."); Pkg.instantiate()

using OceanTurb, PyPlot, PyPlotPlus

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In [ ]:
# First, we define the model.
parameters = KPP.Parameters(K₀=1)
model = KPP.Model(N=200, L=100.0, parameters=parameters);

# No-flux boundary conditions (the default)

When a `KPP.Model` is initialized, it assumes
a default no-flux boundary condition for `U`, `V`, `T`, and `S`.

In [ ]:
# Next we use a simple initial condition and iterate forward
L = model.grid.L
z0, dz = -L/2, L/20
T0(z) = exp(-(z-z0)^2 / 2dz^2)

# Set T to the function T0(z)
model.solution.T = T0

# Initialize plotting
ax, fig = subplots()
xlabel(L"T")
ylabel(L"z")
title("No flux boundary conditions")
cornerspines()

plot(model.solution.T.data, nodes(model.solution.T), "--", label="initial condition")

# Run
dt = 0.01
nt = 1000
ni = 10

for i = 1:ni
    iterate!(model, dt, nt)
    plot(model.solution.T.data, nodes(model.solution.T))
end

legend();

# A constant flux boundary condition

Next, we demonstrate how to set a constant flux boundary condition in `OceanTurb.KPP`.

In [ ]:
# Re-initialize the solution
model.solution.T = 0 # in OceanTurb.jl, this sets all grid values of T to 0.

# Set top flux boundary condition
model.bcs.T.top = FluxBoundaryConition(1.0)
model.bcs.T.bottom = FluxBoundaryConition(-0.01)

# Initialize plotting
ax, fig = subplots()
xlabel(L"T")
ylabel(L"z")
title("Constant flux top boundary condition")
cornerspines()

plot(model.solution.T.data, zdata(model.solution.T), "--", label="initial condition")

# Run
dt = 0.01
nt = 1000
ni = 10

for i = 1:ni
    iterate!(model, dt, nt)
    plot(model.solution.T.data, zdata(model.solution.T))
end

legend();